<a href="https://colab.research.google.com/github/amalvarezme/AprendizajeMaquina/blob/main/7_TopicosAvanzados/4_DKernelsEmbeddings/4_GPFlow_GaussianProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesos Gaussianos y TensorFlow



## Gaussian Process Regressor (GPR)

Un Gaussian Process Regressor (GPR), o regresor basado en procesos Gaussianos, es un método de aprendizaje no paramétrico utilizado para problemas de regresión. A continuación, se presentan los conceptos principales y la matemática fundamental de un GPR:

### Proceso Gaussiano (GP)

- Un Proceso Gaussiano es una generalización de la distribución normal multivariada a dimensiones infinitas.
- En el contexto de la regresión, un GP es un conjunto de variables aleatorias, cualquier subconjunto de las cuales tiene una distribución normal multivariada.
- Un proceso Gaussiano está completamente definido por su función de media $ m(\mathbf{x})$ y su función de covarianza o kernel $k(\mathbf{x}, \mathbf{x}')$:

$
f(\mathbf{x}) \sim \mathscr{GP}(m(\mathbf{x}), k(\mathbf{x}, \mathbf{x}'))
$

donde:

- $m(\mathbf{x}) = \mathbb{E}[f(\mathbf{x})]$ es la media.
- $k(\mathbf{x}, \mathbf{x}') = \mathbb{E}[(f(\mathbf{x}) - m(\mathbf{x}))(f(\mathbf{x}') - m(\mathbf{x}'))]$ es la covarianza o kernel.


### Función de Media y Función de Covarianza

- Función de Media  $m(\mathbf{x})$: Para simplificar, generalmente se asume que la función de media es cero, $ m(\mathbf{x}) = 0 $, aunque se pueden incorporar términos más complejos.

- Función de Covarianza $k(\mathbf{x}, \mathbf{x}')$: Define la similitud entre diferentes puntos de datos. Un kernel comúnmente usado es el kernel RBF (Radial Basis Function) o kernel Gaussiano, definido como:

$k(\mathbf{x}, \mathbf{x}') = \sigma_f^2 \exp\left(-\frac{||\mathbf{x} - \mathbf{x}'||^2}{2l^2}\right)$

donde $\sigma_f^2$ es la varianza y $ l $ es el parámetro de longitud que controla la suavidad de las funciones generadas por el GP.

### Inferencia y Predicción

Dado un conjunto de datos de entrenamiento $ \mathbf{X} = \{\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_n\} $ y sus correspondientes observaciones $ \mathbf{y} = \{y_1, y_2, \ldots, y_n\} $, queremos predecir la salida $ y_* $ para un nuevo punto de entrada $\mathbf{x}_* $.

### Distribución Conjunta

Se asume que los valores de las observaciones $\mathbf{y}$ y el valor predicho $y_*$ siguen una distribución normal conjunta:

\begin{equation}
\begin{bmatrix}
\mathbf{y} \\
y_*
\end{bmatrix}
\sim \mathscr{N}\left(
\begin{bmatrix}
\mathbf{0} \\
0
\end{bmatrix},
\begin{bmatrix}
K(\mathbf{X}, \mathbf{X}) + \sigma_n^2 I & K(\mathbf{X}, \mathbf{x}_*) \\
K(\mathbf{x}_*, \mathbf{X}) & K(\mathbf{x}_*, \mathbf{x}_*)
\end{bmatrix}
\right)
\end{equation}

donde:


- $ K(\mathbf{X}, \mathbf{X})$ es la matriz de covarianza entre los puntos de entrenamiento.
- $ K(\mathbf{X}, \mathbf{x}_*) $ es el vector de covarianza entre los puntos de entrenamiento y el nuevo punto.
- $\sigma_n^2$ es la varianza del ruido.


### Predicción de la Media y la Varianza

Condicionando en los datos observados, la distribución predictiva para $y_*$ es:


$y_* | \mathbf{X}, \mathbf{y}, \mathbf{x}_* \sim \mathscr{N}(\mu_*, \sigma_*^2)$

donde:

### Media Predictiva

$\mu_* = K(\mathbf{x}_*, \mathbf{X}) [K(\mathbf{X}, \mathbf{X}) + \sigma_n^2 I]^{-1} \mathbf{y}$
    
    
### Varianza Predictiva
  
  $\sigma_*^2 = K(\mathbf{x}_*, \mathbf{x}_*) - K(\mathbf{x}_*, \mathbf{X}) [K(\mathbf{X}, \mathbf{X}) + \sigma_n^2 I]^{-1} K(\mathbf{X}, \mathbf{x}_*)$

### Ventajas y Desventajas de GPR

Ventajas:

- Ofrece predicciones probabilísticas (incertidumbre en las predicciones).
- Es flexible y puede modelar funciones complejas.
- No requiere suposiciones específicas sobre la forma de la función de mapeo entre entradas y salidas.


Desventajas:

- Escalabilidad limitada a conjuntos de datos grandes debido a la inversión de matrices ($O(n^3)$ para $n$ puntos de datos).
- La elección del kernel y sus hiperparámetros puede ser crucial para un buen rendimiento.





# Gaussian Process Classification (GPC)

- Para extender el Gaussian Process Regressor (GPR) a un problema de clasificación, utilizamos un enfoque conocido como Gaussian Process Classification (GPC).

- A diferencia de la regresión, donde el objetivo es predecir un valor continuo, en la clasificación se busca predecir una etiqueta discreta (por ejemplo, clases 0 y 1).

### Modelo Latente

- En el GPC, en lugar de modelar la salida  $y$ directamente, modelamos una función latente  $f(\mathbf{x})$ con un proceso Gaussiano y luego pasamos esta función a través de una función sigmoide para obtener probabilidades de clase.

- Para un problema de clasificación binaria:


$f(\mathbf{x}) \sim \mathscr{GP}(m(\mathbf{x}), k(\mathbf{x}, \mathbf{x}')),$

donde, como en el caso de la regresión, $m(\mathbf{x})$ es la función de media (generalmente asumida como cero) y $k(\mathbf{x}, \mathbf{x}')$ es la función de covarianza o kernel.

### Función de Enlace Sigmoide

- Para convertir la función latente $f(\mathbf{x})$ en probabilidades, se utiliza una función de enlace sigmoide, como la función logística:


$\sigma(f) = \frac{1}{1 + e^{-f}}$.

- La probabilidad de pertenecer a la clase 1 dada la función latente $f(\mathbf{x})$ es:


$p(y = 1 \mid f(\mathbf{x})) = \sigma(f(\mathbf{x}))$.

### Inferencia y Predicción

Dado un conjunto de datos de entrenamiento $ \mathbf{X} = \{\mathbf{x}_1, \mathbf{x}_2, \ldots, \mathbf{x}_n\}$ y sus correspondientes etiquetas $\mathbf{y} = \{y_1, y_2, \ldots, y_n\} $, queremos predecir la probabilidad de que una nueva entrada $\mathbf{x}_*$ pertenezca a la clase 1.

### Distribución Conjunta

- La inferencia en GPC es más compleja que en el caso de regresión debido a la naturaleza no lineal de la función de enlace sigmoide. La distribución conjunta de $ \mathbf{f} = \{f(\mathbf{x}_1), \ldots, f(\mathbf{x}_n)\} $ es Gaussiana:

$
\mathbf{f} \sim \mathscr{N}(\mathbf{0}, K(\mathbf{X}, \mathbf{X})),
$

donde $ K(\mathbf{X}, \mathbf{X}) $ es la matriz de covarianza calculada usando el kernel.

### Distribución Posterior

- La distribución posterior de $\mathbf{f} $ dada las observaciones $ \mathbf{y}$ se define como:

$
p(\mathbf{f} \mid \mathbf{X}, \mathbf{y}) = \frac{p(\mathbf{y} \mid \mathbf{f}) p(\mathbf{f} \mid \mathbf{X})}{p(\mathbf{y} \mid \mathbf{X})}.
$

- Dado que la función de enlace sigmoide introduce una no linealidad, $ p(\mathbf{y} \mid \mathbf{f}) $ no es una Gaussiana, lo que hace que la posterior no sea analíticamente tractable.

### Métodos de Aproximación

Para realizar la inferencia, se utilizan métodos de aproximación como:


- Laplace Approximation: Aproxima el posterior como una Gaussiana centrada en el modo de la distribución posterior.
- Expectation Propagation (EP): Una aproximación iterativa que busca minimizar la divergencia de Kullback-Leibler entre la distribución aproximada y la verdadera posterior.
- Variational Inference: Maximiza una cota inferior de la evidencia del modelo para obtener una aproximación Gaussiana de la posterior.


### Ejercicio

Mediante un ejemplo ilustrativo, describa las diferencias conceptuales y matemáticas de los tres métodos de aproximación descritos anteriormente.

### Predicción

- Para una nueva entrada $\mathbf{x}_* $, la predicción se basa en la probabilidad posterior:


$p(y_* = 1 \mid \mathbf{x}_*, \mathbf{X}, \mathbf{y}) = \int \sigma(f_*) p(f_* \mid \mathbf{x}_*, \mathbf{X}, \mathbf{y}) df_*$.

- Este integral se puede resolver analíticamente para algunas aproximaciones, pero generalmente requiere métodos numéricos.

### Ventajas y Desventajas del GPC

Ventajas:

- Proporciona predicciones probabilísticas con estimaciones de incertidumbre.
- Flexible para modelar funciones complejas y no lineales.
- No requiere suposiciones específicas sobre la forma de la función de mapeo.


Desventajas:

- La inferencia es más compleja y computacionalmente costosa que en la regresión.
- Escalabilidad limitada a conjuntos de datos grandes debido a la inversión de matrices ($O(n^3)$ para $n$ puntos de datos).
- La elección del kernel y sus hiperparámetros es crucial para un buen rendimiento.


# GP con tensores : [GPFlow](https://gpflow.github.io/GPflow/2.9.1/index.html)


### Gaussian Process Regression (GPR) con GPflow

En la GPR, modelamos la función subyacente $f(\mathbf{x})$ que genera los datos observados como un proceso Gaussiano. Esto significa que cualquier conjunto finito de puntos de la función tiene una distribución conjunta Gaussiana.

### Implementación de GPR con GPflow

GPflow es una biblioteca basada en TensorFlow diseñada para construir y entrenar modelos de procesos Gaussianos. Permite implementar de manera eficiente modelos de GPR utilizando optimización de máxima verosimilitud marginal (MLE) u otros métodos de optimización.

### Creación del Modelo en GPflow

En GPflow, un modelo GPR se construye especificando un kernel y los datos de entrenamiento. A continuación, se presenta un ejemplo básico de cómo crear un modelo GPR en GPflow:



In [ ]:
!pip install gpflow --upgrade #tensorflow~=2.12.0 tensorflow-probability~=0.20.0

In [ ]:
import tensorflow
import gpflow
import numpy as np

In [ ]:
print(tensorflow.__version__)
print(gpflow.__version__)

In [ ]:

# Datos de entrenamiento
X = np.random.rand(100, 1)  # Características
Y = np.sin(12 * X) + 0.66 * np.cos(25 * X) + 0.1 * np.random.randn(100, 1)  # Observaciones con ruido

# Kernel RBF
kernel = gpflow.kernels.SquaredExponential()

# Modelo GPR
model = gpflow.models.GPR(data=(X, Y), kernel=kernel)
print('kernel hyperparameter initial point\n',model.kernel.lengthscales)

### Optimización del Modelo

Una vez creado el modelo, se puede optimizar usando un optimizador de GPflow. Generalmente, se utiliza el método de máxima verosimilitud marginal (MLE) para ajustar los parámetros del kernel y los hiperparámetros del modelo.


In [ ]:
# Optimización del modelo
opt = gpflow.optimizers.Scipy()
opt.minimize(model.training_loss, model.trainable_variables)

### Predicción con GPflow

- Después de entrenar el modelo, se pueden realizar predicciones para nuevos datos utilizando el modelo ajustado.
- GPflow proporciona métodos para obtener tanto la media predictiva como la varianza, lo cual es útil para estimar la incertidumbre en las predicciones.



In [ ]:
# Nuevos datos para predicción
Xnew = np.linspace(0, 1, 100).reshape(-1, 1)

# Predicción usando el modelo GPR
f_mean, f_var = model.predict_f(Xnew, full_cov=False)
y_mean, y_var = model.predict_y(Xnew)

### Graficar predictiva

In [ ]:
import matplotlib.pyplot as plt

f_lower = f_mean - 1.96 * np.sqrt(f_var)
f_upper = f_mean + 1.96 * np.sqrt(f_var)
y_lower = y_mean - 1.96 * np.sqrt(y_var)
y_upper = y_mean + 1.96 * np.sqrt(y_var)

plt.plot(X, Y, "kx", mew=2, label="input data")
plt.plot(Xnew, f_mean, "-", color="C0", label="mean")
plt.plot(Xnew, f_lower, "--", color="C0", label="f 95% confidence")
plt.plot(Xnew, f_upper, "--", color="C0")
plt.fill_between(
    Xnew[:, 0], f_lower[:, 0], f_upper[:, 0], color="C0", alpha=0.1
)
plt.plot(Xnew, y_lower, ".", color="C0", label="Y 95% confidence")
plt.plot(Xnew, y_upper, ".", color="C0")
plt.fill_between(
    Xnew[:, 0], y_lower[:, 0], y_upper[:, 0], color="C0", alpha=0.1
)
plt.legend()

In [ ]:
print('kernel hyperparameter trained\n',model.kernel.lengthscales)


### Ventajas y Desventajas del GPR con GPflow

Ventajas:

  - Predicciones probabilísticas: GPflow facilita la obtención de predicciones probabilísticas con incertidumbres, lo que es útil para evaluar la confiabilidad de las predicciones.
  - Flexibilidad y eficiencia: La integración con TensorFlow permite aprovechar técnicas de optimización avanzadas y hardware acelerado, como GPUs.
  - Fácil de usar: GPflow tiene una API amigable que simplifica la creación y entrenamiento de modelos de procesos Gaussianos.


Desventajas:

- Escalabilidad limitada: A pesar de las optimizaciones, los modelos de GPR siguen siendo computacionalmente costosos para grandes conjuntos de datos debido a la inversión de matrices.
- Ajuste de hiperparámetros: La selección del kernel adecuado y el ajuste de hiperparámetros son cruciales y pueden requerir experiencia y ajuste manual.


## Ejercicio

- Describa el modelo y la optimización de los GPR, GPC, VGP, SGPR, y SVGP.

- Discuta los métodos de optimización que utiliza GPFlow con scipy y TensorFlow.

- Implemente un clasificador multiclase, para la base de datos fashionMnist, utilizando un Sparse GP implementado con GPflow y optimizador de TensorFlow. (ver [https://gpflow.github.io/GPflow/develop/getting_started.html](https://gpflow.github.io/GPflow/develop/getting_started.html))

- Discuta las ventajas y desventajas de un clasificador tipo autoencoder variacional vs un modelo basado en GP y GPflow.

# 2.Discuta los métodos de optimización que utiliza GPFlow con scipy y TensorFlow.

**GPflow** es una biblioteca de código abierto para la implementación de modelos de Procesos Gaussianos (GP) utilizando TensorFlow. Los modelos de GP son altamente flexibles y proporcionan estimaciones de incertidumbre, pero requieren la optimización de sus hiperparámetros para ajustar adecuadamente los datos.

La optimización en **GPflow** implica ajustar los parámetros del modelo (por ejemplo, los parámetros del kernel, los puntos de inducción en modelos sparse, etc.) para maximizar una función objetivo, como la Evidencia Inferior del Logaritmo Marginal (ELBO) en modelos variacionales.

**Optimización con scipy en GPflow**

* Uso de scipy para Optimización
scipy es una biblioteca de Python que proporciona algoritmos y herramientas matemáticas y científicas. GPflow puede utilizar los optimizadores de scipy para ajustar los parámetros del modelo.

* Métodos de Optimización Disponibles en scipy
Los optimizadores de scipy son métodos clásicos de optimización numérica, principalmente deterministas y basados en gradientes. Algunos de los optimizadores más comunes utilizados con GPflow son:

L-BFGS-B: Un método de Cuasi-Newton limitado que utiliza aproximaciones de la matriz Hessiana. Soporta límites en los parámetros.
CG (Conjugate Gradient): Un método de gradiente conjugado que es eficiente para problemas de gran escala.
BFGS: Método de Cuasi-Newton que no soporta límites en los parámetros.

* Implementación en GPflow

Para utilizar un optimizador de scipy en GPflow, se sigue generalmente este enfoque:

Definir el Modelo: Crear una instancia del modelo GPflow con los datos y la configuración deseada.

Configurar el Optimizador: Utilizar gpflow.optimizers.Scipy() para acceder al optimizador de scipy.

Ejecutar la Optimización: Llamar al método minimize() pasando la función objetivo (por ejemplo, el negativo del log-marginal likelihood) y las variables a entrenar.

* Ventajas y Limitaciones

*Ventajas:*

Convergencia Rápida: Los métodos de scipy suelen converger en menos iteraciones debido a su naturaleza determinista y uso de información de segundo orden.

Adecuado para Conjuntos de Datos Pequeños: Son ideales para problemas donde los datos pueden ser procesados en memoria y el cálculo de gradientes completos es factible.

*Limitaciones:*

No Escalable a Grandes Conjuntos de Datos: El cálculo de gradientes en cada iteración sobre todos los datos es computacionalmente costoso para grandes conjuntos.

No Soporta Minibatching: Los optimizadores de scipy no están diseñados para trabajar con lotes pequeños de datos.

# 4.Discuta las ventajas y desventajas de un clasificador tipo autoencoder variacional vs un modelo basado en GP y GPflow.

**Autoencoder Variacional (VAE)**
Un Autoencoder Variacional es un tipo de modelo generativo que aprende una representación latente (codificación) de los datos de entrada. Consiste en dos partes principales:

Codificador (Encoder): Mapea los datos de entrada a una distribución latente.
Decodificador (Decoder): Reconstruye los datos de entrada a partir de muestras de la distribución latente.
Los VAEs son entrenados para minimizar la diferencia entre los datos de entrada y su reconstrucción, al mismo tiempo que regulan la estructura de la representación latente para que siga una distribución conocida (normalmente una distribución normal multivariada).

En el contexto de clasificación, los VAEs pueden ser extendidos para realizar tareas de clasificación incorporando variables latentes que capturan la información relevante para las clases.


**Proceso Gaussiano (GP) con GPflow**
Un Proceso Gaussiano es un modelo probabilístico no paramétrico utilizado para regresión y clasificación. Un GP define una distribución sobre funciones y proporciona estimaciones de incertidumbre en las predicciones.

GPflow es una biblioteca de código abierto basada en TensorFlow que facilita la implementación y entrenamiento de modelos de Procesos Gaussianos, incluyendo extensiones para manejar grandes conjuntos de datos y realizar clasificación multiclase.


**Ventajas del Clasificador Basado en VAE**

* Aprendizaje de Representaciones Latentes
Captura de Estructuras Complejas: Los VAEs aprenden una representación latente de los datos que puede capturar estructuras complejas y variaciones en los datos de entrada.
Dimensionalidad Reducida: La representación latente suele ser de dimensionalidad más baja, lo que facilita el análisis y la visualización.

* Capacidad Generativa
Generación de Nuevos Datos: Al modelar la distribución de los datos, los VAEs pueden generar nuevas muestras similares a las del conjunto de entrenamiento.
Interpolación en el Espacio Latente: Permite explorar transiciones suaves entre diferentes clases o atributos.

* Flexibilidad en Modelado 
Extensiones y Personalización: Los VAEs pueden ser adaptados para tareas específicas, como modelos condicionados o variantes que incorporan información de clase durante el entrenamiento.

* Manejo de Datos de Alta Dimensionalidad
Escalabilidad: Los VAEs son adecuados para trabajar con datos de alta dimensionalidad, como imágenes, audio y texto.


**Desventajas del Clasificador Basado en VAE**

***Complejidad del Entrenamiento***

* Optimización Desafiante: El entrenamiento de VAEs implica optimizar una función de pérdida que combina términos de reconstrucción y regularización (divergencia KL), lo que puede ser complejo.

* Sensibilidad a la Configuración: Requiere un ajuste cuidadoso de hiperparámetros, como la arquitectura de la red, tasas de aprendizaje y términos de regularización.

**Calidad de las Representaciones Latentes**

* Limitaciones en la Separación de Clases: Las representaciones latentes pueden no separar claramente las clases sin modificaciones adicionales, lo que puede afectar el rendimiento de clasificación.

* Necesidad de Información de Clase: Para mejorar la capacidad de clasificación, a menudo es necesario incorporar información de clase durante el entrenamiento (por ejemplo, VAE semisupervisados o VAE condicionales).

* Ausencia de Estimaciones de Incertidumbre. Incertidumbre en Predicciones: Aunque los VAEs son modelos probabilísticos, no proporcionan estimaciones de incertidumbre en las predicciones de clase de manera directa.

* Coste Computacional. Requerimientos de Hardware: Entrenar VAEs profundos puede ser computacionalmente intensivo y requerir hardware acelerado (como GPUs).

**Ventajas del Modelo Basado en GP y GPflow**

**Estimaciones de Incertidumbre**

* Predicciones Probabilísticas: Los GPs proporcionan distribuciones de probabilidad sobre las predicciones, ofreciendo estimaciones de incertidumbre que son valiosas en muchas aplicaciones.

* Detección de Outliers: La incertidumbre permite identificar entradas fuera de la distribución de entrenamiento.


**No Paramétrico y Flexibilidad**

* Adaptabilidad: Los GPs son modelos no paramétricos que pueden adaptarse a diferentes complejidades en los datos sin necesidad de especificar una estructura fija.

* Kernels Personalizables: La elección de diferentes kernels permite capturar relaciones específicas en los datos.

**Interpretabilidad**

* Transparencia en las Predicciones: Es más fácil interpretar cómo el modelo hace predicciones basándose en la similitud entre puntos de datos.


**Implementación con GPflow**

* Facilidad de Uso: GPflow simplifica la implementación de GPs utilizando TensorFlow, aprovechando las capacidades de autodiferenciación y optimización.

* Extensiones para Escalabilidad: Soporta métodos para manejar conjuntos de datos más grandes, como Procesos Gaussianos Sparsos y Minibatching.

**Desventajas del Modelo Basado en GP y GPflow**

**Escalabilidad**

* Coste Computacional: El entrenamiento y predicción con GPs estándar tienen una complejidad cúbica respecto al número de muestras, lo que limita su aplicación en conjuntos de datos grandes.

* Necesidad de Aproximaciones: Para conjuntos de datos grandes, es necesario utilizar aproximaciones (como GPs sparsos), lo que puede afectar la precisión.


**Dificultades en Alta Dimensionalidad**

* Eficiencia en Altas Dimensiones: Los GPs pueden tener dificultades para manejar datos con muchas características de entrada, ya que la eficacia de los kernels puede disminuir.


* Curse of Dimensionality: A medida que aumenta la dimensionalidad, puede ser más difícil para el modelo aprender relaciones significativas.

**Selección y Ajuste de Kernels**

* Dependencia en el Kernel: El rendimiento del GP depende en gran medida de la elección del kernel y sus hiperparámetros.

* Ajuste Complejo: Encontrar el kernel y los hiperparámetros adecuados puede requerir experiencia y tiempo.

**Limitaciones en Modelos Profundos**

* Representación de Características Complejas: Aunque es posible combinar GPs con redes neuronales (Deep GPs), esto agrega complejidad y aún es un área de investigación activa.


**Conclusión**

La elección entre un clasificador basado en Autoencoder Variacional y un modelo de Proceso Gaussiano con GPflow depende de varios factores, incluyendo el tamaño y la dimensionalidad de los datos, la necesidad de estimaciones de incertidumbre, la capacidad generativa y la interpretabilidad.

El **VAE** es más adecuado para:

* Datos de alta dimensionalidad y complejidad.
* Aplicaciones que requieren generación de nuevos datos o aprendizaje de representaciones latentes.
* Escenarios donde se dispone de grandes cantidades de datos y se puede aprovechar el poder de las redes neuronales profundas.

El modelo basado en **GP y GPflow** es más adecuado para:

* Situaciones donde las estimaciones de incertidumbre son cruciales.
* Conjuntos de datos más pequeños donde se puede aprovechar la capacidad de los GPs sin incurrir en costos computacionales prohibitivos.
* Problemas que se benefician de la interpretabilidad y la naturaleza bayesiana de los GPs.